# Main

In Python, there are various libraries for working with SQL databases. These include the following:

| Database | Library | Description | Advantages | Disadvantages |
| - | - | - | - | - |
| SQLite | `sqlite3` | SQLite databases are serverless and self-contained (read and write data to a file), meaning that you don't need to install and run a server to perform database operations. | You can create a serverless database on local machine. Lightweight library. Part of standard Python libraries - no need to install. | |
| MySQL | No default python sql module to connect to a sql database. e.g. `mysql-connector-python` | | | Need to connect to external server. |
| PostgreSQL | same as mysql. E.g. `psycopg2` | | | Need to connect to external server. |


# SQL

## SQLite3

In [ ]:
import sqlite3

# Create connection
connection = sqlite3.connect('output/tutorial.db')
# Create cursor object
cursor = connection.cursor()

cursor.execute("""CREATE TABLE gta (
    release_year integer, 
    release_name text, 
    city text
    )""")

release_list = [
    (1997, "Grand Theft Auto", "state of New Guernsey"),
    (1999, "Grand Theft Auto 2", "Anywhere, USA"),
    (2001, "Grand Theft Auto III", "Liberty City"),
    (2002, "Grand Theft Auto: Vice City", "Vice City"),
    (2004, "Grand Theft Auto: San Andreas", "state of San Andreas"),
    (2008, "Grand Theft Auto IV", "Liberty City"),
    (2013, "Grand Theft Auto V", "Los Santos")
]

# Three placeholders for the three items in tuples in the list
# IMPORTANT: use "?" to avoid SQL injection attacks
cursor.executemany("insert into gta values (?,?,?)", release_list)

a,b,c = 2015, "Newest GTA", "Mexico"
cursor.execute("INSERT INTO gta VALUES (?,?,?)", (a,b,c))

connection.commit()

# Terminate the connection
connection.close()

In [ ]:
import sqlite3
from sqlite3 import Error

def create_connection(path):
	connection = None
	try:
		# Create connection
		connection = sqlite3.connect(path)
		print('Connection to SQLite db successful!')
	except Error as e:
		print(f"The error {e} occured!")
	return connection

connection = create_connection('output/tutorial.db')
# Create cursor object
cursor = connection.cursor()

# Print table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

# Print column names
data = cursor.execute("SELECT * FROM gta")
for i in data.description:
	print(i[0])

# Print database rows
for i in cursor.execute("select * from gta"):
	print(i)

# Print specific rows
for i in cursor.execute("select * from gta where city='Liberty City'"):
    print(f" - {i}")

cursor.execute("SELECT * FROM gta WHERE city='Liberty City'")
print( cursor.fetchall() )

cursor.execute("SELECT * FROM gta WHERE release_year=?", (1999,))
print( cursor.fetchall() )

# Terminate the connection
connection.close()

Connection to SQLite db successful!
[('gta',)]
release_year
release_name
city
(1997, 'Grand Theft Auto', 'state of New Guernsey')
(1999, 'Grand Theft Auto 2', 'Anywhere, USA')
(2001, 'Grand Theft Auto III', 'Liberty City')
(2002, 'Grand Theft Auto: Vice City', 'Vice City')
(2004, 'Grand Theft Auto: San Andreas', 'state of San Andreas')
(2008, 'Grand Theft Auto IV', 'Liberty City')
(2013, 'Grand Theft Auto V', 'Los Santos')
(2015, 'Newest GTA', 'Mexico')
 - (2001, 'Grand Theft Auto III', 'Liberty City')
 - (2008, 'Grand Theft Auto IV', 'Liberty City')
[(2001, 'Grand Theft Auto III', 'Liberty City'), (2008, 'Grand Theft Auto IV', 'Liberty City')]
[(1999, 'Grand Theft Auto 2', 'Anywhere, USA')]


## MySQL

```py
import mysql.connector
import sshtunnel
import csv

sshtunnel.SSH_TIMEOUT = 5.0
sshtunnel.TUNNEL_TIMEOUT = 5.0

with sshtunnel.SSHTunnelForwarder(
    ('your SSH hostname'),
    ssh_username='your PythonAnywhere username', ssh_password='the password you use to log in to the PythonAnywhere website',
    remote_bind_address=('your PythonAnywhere database hostname, eg. yourusername.mysql.pythonanywhere-services.com', 3306)
) as tunnel:
    connection = mysql.connector.connect(
        user='your PythonAnywhere database username',
        passwd='your PythonAnywhere database password',
        host='127.0.0.1', port=tunnel.local_bind_port,
        db='your database name, eg yourusername$mydatabase',
    )
    # Do stuff
	mycursor = connection.cursor()
	mycursor.execute("SELECT * FROM Persons;")
	a = mycursor.fetchall()
	print(a)
    # Save the SQL query
    with open('output/sql_query.csv', 'w') as fp:
		csvwriter = csv.writer(fp, delimiter=',', lineterminator='\n')
		csvwriter.writerows(a)
	mycursor.execute("INSERT INTO Persons (first_name, last_name, age) VALUES ('Name', 'Surname', 25);")
    connection.commit()
    # close connection
    connection.close()
```

## PostgreSQL

<u>First way to connect to the database</u>:
- Need to close `conn` and `cur`;
- Need to commit the changes;
```py
import psycopg2

conn, cur = None, None

try:
    conn = psycopg2.connect(
        host='localhost',
        user='postgres',
        password='...',
        database='database1'
    )
    cur = conn.cursor()
    # SQL statements
    conn.commit()

except Exception as error:
    print(error)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:
        conn.close()

```

<u>Second way to connect</u>:
- No need to close `cur`, because it's open with an `if` clause
- Still DO need to close `conn`
- No need to commit
```py
import psycopg2

conn = None

try:
    with psycopg2.connect(
        host='localhost',
        user='postgres',
        password='...',
        database='database1'
    ) as conn:
        with conn.cursor() as cur:
            cur = conn.cursor()
            # SQL statements

except Exception as error:
	print(error)
finally:
	if conn is not None:
		conn.close()

```

---

Use this option if you want to return data in form of a dictionary:
```py
import psycopg2.extras

cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)

cur.execute('SELECT * FROM employee;')
# print(cur.fetchall())
### Can also do this:
for i in cur.fetchall():
	print(i)
	print(i['name'], i['salary'])
```

---


```py
# Some SQL statements below
cur.execute('SELECT version()')
row = cur.fetchone()
print(row)

cur.execute('SELECT * FROM table1;')
rows = cur.fetchall()
print(rows)


cur.execute('DROP TABLE IF EXISTS employee;')


create_script = '''CREATE TABLE IF NOT EXISTS employee (
	id INT PRIMARY KEY, 
	name VARCHAR(40) NOT NULL, 
	salary INT, 
	dept_id VARCHAR(30));'''
cur.execute(create_script)


insert_script = """INSERT INTO employee (id, name, salary, dept_id) VALUES (%s, %s, %s, %s)"""
### Insert one value
insert_value = (1, 'James', 12000, 'D1')
cur.execute(insert_script, insert_value)
### Insert multiple values
insert_values = [(1, 'James', 12000, 'D1'), (2, 'Robin', 15000, 'D1'), (3, 'Jane', 20000, 'D2')]
for i in insert_values:
	cur.execute(insert_script, i)


update_script = """UPDATE employee SET salary = salary + (salary * 0.5)"""
cur.execute(update_script)


delete_script = """DELETE FROM employee WHERE name = %s"""


cur.execute('SELECT * FROM employee;')
# print(cur.fetchall())
### Can also do this:
for i in cur.fetchall():
	print(i)
	print(i['name'], i['salary'])

```



In [ ]:
# Sample code

import psycopg2

try:
	connection = psycopg2.connect('postgres://lotsOfStringsHere')
	print('Connection successful!')
	cursor = connection.cursor()
	cursor.execute("CREATE TABLE IF NOT EXISTS test (id serial PRIMARY KEY, num integer, data varchar);")
	cursor.execute("INSERT INTO test (num, data) VALUES (%s, %s)", (100, "abc'def"))
	cursor.execute("INSERT INTO test (num, data) VALUES (%s, %s)", (200, "second_entry"))
	cursor.execute("SELECT * FROM test;")
	text = cursor.fetchall()
	print(text)
	with open('output/sql_query2.csv', 'w') as fp:
		csvwriter = csv.writer(fp, delimiter=',', lineterminator='\n')
		csvwriter.writerows(text)
	connection.commit()
except Exception as exception:
	print(exception)
finally:
	connection.close()
	print('Connection closed.')

Connection successful!
[(1, 100, "abc'def"), (2, 200, 'second_entry'), (3, 100, "abc'def"), (4, 200, 'second_entry'), (5, 100, "abc'def"), (6, 200, 'second_entry')]
Connection closed.


# noSQL

## MongoDB

performing queries in `pymongo`: https://www.analyticsvidhya.com/blog/2020/08/query-a-mongodb-database-using-pymongo/

https://studio3t.com/knowledge-base/articles/sql-query/

You can write queries in several ways:
```py
### METHOD 1
query = {}
query["fecha"] = {
    u"$gt": datetime(2023,6,12, 16,4)
}
### METHOD 2
query = {
    'camara': 'test_camera',
    'nave': 'test_nave',
    'fecha': { '$gt': datetime.datetime(2023,6,12, 16,4,58) }
}
```

**Studio 3T** is a MongoDB client application for viewing MongoDB. 

Queries there look like this (e.g. if you have a pro version with SQL support):
- `select * from [cam-events] where fecha > date('20230612');`
- `select * from [cam-events] where fecha > date('2023-06-12T16:05:00.000Z');`
- `select * from [cam-events] where fecha > date('20230612') and fecha < date('20230613');`
- Filter one day only: `select * from [cam-events] where fecha >= date('20230612') and fecha < date('20230613');`



### Examples view

**Connect to a db table**
```py
from pymongo import MongoClient
import datetime

connection_uri = "mongodb://a-super-long-string-follows-here@ip:port"
db_name = 'dbname'
table_name = 'tablename'

### You can connect and then close a connection:
client = MongoClient(connection_uri)  
db = client[db_name]  
user_collection = db[table_name]  
query = { 'fecha': { '$gt': datetime.datetime(2023,6,12, 16,5) } }
output = []
cursor = user_collection.find(query)
try:
    for doc in cursor:
        print(doc)
        print(doc['fecha'])
        output.append(doc)
finally:
    client.close()


### PREFERRED METHOD
### You can also connect with context manager, automatically closing whatever connection was present:
with MongoClient(connection_uri) as client:
    db = client[db_name]
    user_collection = db[table_name]
    query = {'date': {'$gt': datetime.datetime(2023,6,13)}}
    cursor = user_collection.find(query)
    output = []
    try:
        for doc in cursor:
            print(doc)
            print(doc['date'])
            output.append(doc)
    except Exception as e:
        print(e)
```

---

**Query examples**

```py
query = {'fecha': { '$gt': datetime.datetime(2023,6,13) }}
query = { 'fecha': { '$gt': datetime.datetime(2023,6,12, 16,5) } }
query = {
    'fecha': {
        '$gte': datetime.datetime(2023,6,12, 16,2),
        '$lt': datetime.datetime(2023,6,12, 16,4)
    }
}
query = {
    'camara': 'test_camera',
    'nave': 'test_nave',
    'fecha': { '$gt': datetime.datetime(2023,6,12, 16,4,58) }
}
```

Also you can read a table from a noSQL database with nested variables into a Pandas dataframe with the following code:

```py
import pymongo
import pandas as pd

with pymongo.MongoClient(connection_uri) as client:
    db = client['database-name']
    user_collection = db['table-name']
    cursor = user_collection.find({}) # get all records
    df = pd.json_normalize(cursor) # this function flattens the nested variables

df
```


### Examples insert

```py
### Insert one
### Example 1 of query
query = {'variable1':1, 'variable2':2}
### Example 2 of query
query = {
    'variable1':1, 
    'runtime': {
        'runtime1': 10, 
        'runtime2':20
    }
}

with MongoClient(connection_uri) as client:
    db = client['databaseName']
    user_collection = db['tableName']
    user_collection.insert_one(query)


### Insert many
query = [
    {'variable1':1, 'variable2'2},
    {'variable1':10, 'variable2':20}
]

with MongoClient(connection_uri) as client:
    db = client['databaseName']
    user_collection = db['tableName']
    user_collection.insert_many(query)

```

---

You can insert something as date, as long as you pass it in format `datetime.datetime.now()` or `datetime.datetime(2023,6,16)`



### Examples update

**Just one condition WHERE**
```py
where = {
    'variable1_date': {'$gte': someDate}
}
replace_with = {'$set': {
    'variable2': 100,
    'variable3': 150
    }
}
user.collection.update_many(
    where, replace_with
)
```

**Many conditions WHERE**
```py
    where={
        'variable1': 'b',
        'variable2': 'b'
    },
    replace_with={
        '$set': {
            'variable3': 'UPDATED AGAIN'
        }
    }
```


### Examples delete

**Delete from collection (table)**
```py
# Upon condition
user_collection.delete_many({'a': '12'})
# Clear collection (delete all records from a table)
user_collection.delete_many({})
```


## ArangoDB

```py
# pip install python-arango --upgrade
from arango import ArangoClient
import pandas as pd

client = ArangoClient(hosts='http://localhost:8529')
db = client.db('<database-id>', username='<username>', password='<username>')

q= """
FOR doc IN students RETURN doc
"""

cur = db.aql.execute(q)
df = pd.DataFrame(cur)
df
```
